# SILVER LAYER SCRIPT

### DATA ACCESS USING APP

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
service_credential = dbutils.secrets.get(scope="<secret-scope>",key="<service-credential-key>")

spark.conf.set("fs.azure.account.auth.type.<storage-account>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account>.dfs.core.windows.net", "<application-id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account>.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory-id>/oauth2/token")

### DATA LOADING

### Reading Data

In [0]:
df_cal = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://bronze@azdedatalake.dfs.core.windows.net/AdventureWorks_Calendar")


In [0]:
df_cust = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://bronze@azdedatalake.dfs.core.windows.net/AdventureWorks_Customers")

In [0]:
df_prod_cat = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://bronze@azdedatalake.dfs.core.windows.net/AdventureWorks_Product_Categories")

In [0]:
df_prod = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://bronze@azdedatalake.dfs.core.windows.net/AdventureWorks_Products")

In [0]:
df_ret = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://bronze@azdedatalake.dfs.core.windows.net/AdventureWorks_Returns")

In [0]:
df_sal = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://bronze@azdedatalake.dfs.core.windows.net/AdventureWorks_Sales*")

In [0]:
df_prod_ter = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://bronze@azdedatalake.dfs.core.windows.net/AdventureWorks_Territories")

In [0]:
df_prod_sub = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://bronze@azdedatalake.dfs.core.windows.net/Product_Subcategories")

### Transformations


In [0]:
df_cal = df_cal.withColumn('Month', month(col('Date')))\
    .withColumn('Year',year(col('Date')))



In [0]:
df_cal.write.format("parquet") \
    .mode("append") \
    .option("path", "abfss://silver@azdedatalake.dfs.core.windows.net/Calendar") \
    .save()


In [0]:
df_cust = df_cust.withColumn("Name", concat_ws(" ", col("prefix"), col("FirstName"), col("LastName")))



In [0]:
df_cust.write.format("parquet") \
    .mode("append") \
    .option("path", "abfss://silver@azdedatalake.dfs.core.windows.net/Customers") \
    .save()

In [0]:
df_prod_sub.write.format("parquet") \
    .mode("append") \
    .option("path", "abfss://silver@azdedatalake.dfs.core.windows.net/Product_Subcategories") \
    .save()

In [0]:


df_prod = df_prod.withColumn("ProductSKU", split(col("ProductSKU"), "-").getItem(0))\
             .withColumn("ProductName", split(col("ProductName"), " ").getItem(0))

In [0]:
df_prod.write.format("parquet") \
    .mode("append") \
    .option("path", "abfss://silver@azdedatalake.dfs.core.windows.net/Products") \
    .save()

In [0]:
df_ret.write.format("parquet") \
    .mode("append") \
    .option("path", "abfss://silver@azdedatalake.dfs.core.windows.net/Returns") \
    .save()

In [0]:
df_prod_ter.write.format("parquet") \
    .mode("append") \
    .option("path", "abfss://silver@azdedatalake.dfs.core.windows.net/Product_Territories") \
    .save()

In [0]:
df_sal = df_sal.withColumn('StockDate',to_timestamp('StockDate'))

In [0]:
df_sal = df_sal.withColumn('OrderNumber',regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_sal = df_sal.withColumn('TotalQuantity',col('OrderLineItem')*col('OrderQuantity'))

In [0]:
df_sal.write.format("parquet") \
    .mode("append") \
    .option("path", "abfss://silver@azdedatalake.dfs.core.windows.net/Sales") \
    .save()